In [310]:
%reload_ext autoreload
%autoreload 2
from datasets import load_dataset


# ignore_verifications=Trueをつけないとエラーとなる
oscar_dataset = load_dataset('oscar', 'unshuffled_original_ja', 
                       split='train', 
                       ignore_verifications=True,
                       streaming=True)
#mc4の場合はこちら
mc4_dataset = load_dataset('mc4', 'ja',split='train', streaming=True)
dataset=oscar_dataset

/home/user/miniconda3/envs/crawl/lib/python3.12/site-packages/datasets/load.py:2516: FutureWarning: 'ignore_verifications' was deprecated in favor of 'verification_mode' in version 2.9.1 and will be removed in 3.0.0.
You can remove this warning by passing 'verification_mode=no_checks' instead.
  warnings.warn(
/home/user/miniconda3/envs/crawl/lib/python3.12/site-packages/datasets/load.py:1454: FutureWarning: The repository for oscar contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/oscar
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
/home/user/miniconda3/envs/crawl/lib/python3.12/site-packages/datasets/load.py:1454: FutureWarning: The repository for mc4 contains custom code which must be executed to correctly load the dataset. 

In [416]:
from src.cleaner.auto_cleaner import clean_text
from src.classifier.problem_generator import naive_generate_questions
from tqdm import tqdm

#データセットからシンプルに問題を生成する
naive_generate_questions(dataset)

300it [00:04, 68.08it/s] 


output to annotations/just_texts.txt


In [313]:
#こちらのサイトでアノテーションを行う good/bad
#https://konbraphat51.github.io/Text-Labeling/

In [389]:
from src.classifier.loader import load_annotated_text
from src.cleaner.parts_filter import parts_count
from src.classifier.text_characterizer import annotation_data_to_X_y,texts_to_X
import json

In [318]:
from src.classifier.word2vec import Word2Vec
w2v=Word2Vec('../data/fasttext/cc.ja.300.bin')


In [474]:
vec,y,annot_texts=annotation_data_to_X_y(w2v,annotation_data_path="annotations/original_data/annotations.txt")
print(vec.shape)

(1676, 302)


In [475]:
from src.classifier.classifier import prepare_tfid_classifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
#pipeline=prepare_pipeline()

pipeline = prepare_tfid_classifier()
# データセットの分割
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train, X_test, y_train, y_test = train_test_split(annot_texts, y, test_size=0.1)

# モデルの訓練
pipeline.fit(X_train, y_train)

# モデルの評価
predictions = pipeline.predict(X_test)
print(classification_report(y_test, predictions))

/home/user/miniconda3/envs/crawl/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


              precision    recall  f1-score   support

           0       0.79      0.75      0.77        79
           1       0.78      0.82      0.80        89

    accuracy                           0.79       168
   macro avg       0.79      0.78      0.78       168
weighted avg       0.79      0.79      0.79       168



In [476]:
#pipeline.fit(X, y)

cnt=0

#アノテーションされたデータを使ってモデルを評価する
for record in dataset:
    cnt+=1
    if cnt>100:
        break
    text=record['text']
    text=clean_text(text)
    if text=="":
        continue
    #pred=pipeline.predict(texts_to_X(w2v,[text]))
    pred=pipeline.predict([text])
    print(pred,text[:300])

[0] 神社などへ一緒に同行して、様々な角度のショットで家族写真やお子様の写真を撮影致します!お好みに合わせて様々な写真を取ることができますので、その場でカメラマンへのリクエストも可能です!お子様の晴れ姿を、緊張していない自然な笑顔で残しませんか?
※七五三の対象者が1名のプランです。ご兄弟、ご友人様もご一緒になどと、主役が2名、3名と複数名いらっしゃる場合は、+5,000円(税抜)/1名の別途料金が発生いたします。
※撮影神社、宮社によっては撮影許可料(3,000円ほど)が別途発生いたす場所がございます。こちらは弊社撮影料金には含まれておりませんので、事前に撮影希望神社へお客様よりご確認、お支払いを
[0] ◎正当な理由による書き込みの削除について:1レス¥5000円1スレ¥20000円の技術作業料が発生します。一回分だけの料金で当方管理下の全サイトで作業が実施されます。支払い方法はAmazonギフト券番号。連絡先は当サイトの登録emailへ。
[0] 騒音注意され腹立てた男、男性の顔殴り眼球破裂:社会:YOMIURI ONLINE(読売新聞)http://www.yomiuri.co.jp/national/news/20131106-OYT1T00269.htm怖い
[1] 名前使い回し効果も使いまわしの数値変更っていうのがなんかネタ切れ感感じますね・・・アンリミテッドで旧カードと合わせて6枚同名キャラが入ったファンデッキ作るのは面白そうですね旧カードと同カード扱いされて合計3枚制限がかかるかは心配ですが・・・
[1] たまには、人を語らせて下さい。近野淳一さんと知り合ったのはここ数年の事、とある方(⬅すばらし方です)の紹介でお店に一緒に来たのがキッカケ。その後、交流があり昨年は3本もお店でライブをやって頂き、盛り上がったよな～と!更新しながらも思い出してしまいますが...ほんとに毎回近野淳一ワールドで、お客様の笑顔や喜ぶ姿が今でも目に焼き付いております。
さんび～なや(_ゝ`)本日は一段と冷え込みも激しく鼻っぱしらが氷りそう。さて、先月は秋田朝日放送さんの番組、人情食堂に出させて頂きました。正直、緊張しましたよ～テレビですよ、人生初めての体験!カメラを前にすると素顔がでてしまいそうで、普段からの店長としての
[1] 岩手県出身のデザイナー、村上雄一郎さんの革

In [477]:
from src.classifier.problem_generator import generate_uncertain_questions
#自信のないデータを問題にする
#generate_uncertain_questions(dataset,w2v,pipeline,annot_texts=annot_texts,start_num=3000)
generate_uncertain_questions(dataset,pipeline,annot_texts=annot_texts,start_num=3000)

0it [00:00, ?it/s]

32109it [00:24, 1306.29it/s]

output to annotations/original_data/uncertain_questions.txt
